In [2]:
import pandas as pd
import numpy as np

import optuna

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten,  Dropout, SpatialDropout2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Nadam

from sklearn.metrics import roc_auc_score

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing import image

In [3]:
tf.__version__

'2.20.0'

In [47]:
data = pd.read_csv(r'C:\Users\admin\Downloads\application.csv')

In [48]:
data.drop(data[['SK_ID_CURR', 'ORGANIZATION_TYPE']], axis = 1, inplace = True)

In [49]:
data['AGE'] = (-data['DAYS_BIRTH'] / 365).astype(int)
data = data.drop(['DAYS_BIRTH'], axis= 1)

In [50]:
segment_mapping = {
    "Lower secondary": 1,
    "Secondary / secondary special": 2,
    "Incomplete higher": 3,
    "Higher education": 4,
    "Academic degree": 5,
}
data["NAME_EDUCATION_TYPE"] = data["NAME_EDUCATION_TYPE"].map(segment_mapping)

In [ ]:
data['CODE_GENDER'] = data['CODE_GENDER'].replace('XNA', np.nan)

In [ ]:
threshold = 0.3
cols_to_drop = data.columns[data.isnull().sum() / len(data) > threshold].tolist()

cols_to_drop

In [ ]:
data.drop(columns=cols_to_drop, inplace=True)

In [57]:
for i in data.columns:
    if data[i].dtype == 'object':
        data[i].fillna(data[i].mode()[0], inplace=True)
    else:
        data[i].fillna(data[i].mean(), inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_12352\1157204993.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[i].fillna(data[i].mean(), inplace=True)


In [54]:
from sklearn.preprocessing import LabelEncoder
encoders = {}
for i in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[i] = le.fit_transform(data[i])
    encoders[i] = le

In [59]:
for i in data:
    print(data[i].isnull().sum())
    

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [61]:
data = pd.get_dummies(data, drop_first=True)

data

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE
0,1,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,1.000000,25
1,0,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,45
2,0,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,52
3,0,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974,52
4,0,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0,1,0,0,0,157500.0,254700.0,27558.0,225000.0,...,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974,25
307507,0,0,0,0,1,0,72000.0,269550.0,12001.5,225000.0,...,0,0,0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974,56
307508,0,0,0,0,1,0,153000.0,677664.0,29979.0,585000.0,...,0,0,0,1.000000,0.000,0.000000,1.000000,0.000000,1.000000,41
307509,1,0,0,0,1,0,171000.0,370107.0,20205.0,319500.0,...,0,0,0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,32


In [63]:
targets = data['TARGET']

inputs = data.drop(['TARGET'],axis=1)

In [65]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

scaled = scaler.transform(inputs)

inputs_scaled = pd.DataFrame(scaled, columns=inputs.columns)

inputs_scaled

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE
0,-0.324395,1.388015,-0.717914,0.664531,-0.577538,0.142129,-0.478095,-0.166149,-0.507465,0.435227,...,-0.024402,-0.022529,-0.018305,-8.210023e-02,-0.067957,-1.805048e-01,-0.313873,-3.594746e-01,-5.176655e-01,-1.542169
1,-0.324395,-0.720370,-0.717914,-1.504820,-0.577538,0.426792,1.725450,0.592677,1.600698,-2.380655,...,-0.024402,-0.022529,-0.018305,-8.210023e-02,-0.067957,-1.805048e-01,-0.313873,-3.594746e-01,-1.092866e+00,0.130831
2,3.082659,1.388015,1.392925,0.664531,-0.577538,-0.427196,-1.152888,-1.404676,-1.092389,0.435227,...,-0.024402,-0.022529,-0.018305,-8.210023e-02,-0.067957,-1.805048e-01,-0.313873,-3.594746e-01,-1.092866e+00,0.716381
3,-0.324395,-0.720370,-0.717914,0.664531,-0.577538,-0.142533,-0.711430,0.177869,-0.653696,0.435227,...,-0.024402,-0.022529,-0.018305,3.336720e-17,0.000000,-3.645032e-17,0.000000,-7.516682e-17,-3.831603e-16,0.716381
4,-0.324395,1.388015,-0.717914,0.664531,-0.577538,-0.199466,-0.213734,-0.361755,-0.068772,0.435227,...,-0.024402,-0.022529,-0.018305,-8.210023e-02,-0.067957,-1.805048e-01,-0.313873,-3.594746e-01,-1.092866e+00,0.883681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,-0.324395,1.388015,-0.717914,-1.504820,-0.577538,-0.047646,-0.855489,0.031009,-0.848671,0.435227,...,-0.024402,-0.022529,-0.018305,3.336720e-17,0.000000,-3.645032e-17,0.000000,-7.516682e-17,-3.831603e-16,-1.542169
307507,-0.324395,-0.720370,-0.717914,0.664531,-0.577538,-0.408219,-0.818594,-1.042339,-0.848671,0.435227,...,-0.024402,-0.022529,-0.018305,3.336720e-17,0.000000,-3.645032e-17,0.000000,-7.516682e-17,-3.831603e-16,1.050981
307508,-0.324395,-0.720370,-0.717914,0.664531,-0.577538,-0.066623,0.195379,0.198050,0.126202,0.435227,...,-0.024402,-0.022529,-0.018305,1.274116e+01,-0.067957,-1.805048e-01,0.859944,-3.594746e-01,-5.176655e-01,-0.203769
307509,-0.324395,-0.720370,-0.717914,0.664531,-0.577538,0.009287,-0.568757,-0.476324,-0.592767,0.435227,...,-0.024402,-0.022529,-0.018305,-8.210023e-02,-0.067957,-1.805048e-01,-0.313873,-3.594746e-01,-1.092866e+00,-0.956619


In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=42)

In [89]:
def create_model(trial):
    # Building artificial neural network
    model = Sequential()

     # we add 2 hidden layers and 1 output layer
    model.add(Dense(units=trial.suggest_int('units_layer1', 6, 32), activation='relu'))
    model.add(Dense(units=trial.suggest_int('units_layer2', 6, 32), activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    # Suggest hyperparameters for the optimizer
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adagrad'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'adagrad':
        optimizer = Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])
    
    return model

In [105]:
from tensorflow.keras.callbacks import EarlyStopping
    
def optimal(trial):
    
    # Suggest the number of epochs and batch size
    epochs = trial.suggest_int('epochs', 10, 100)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    
    model = create_model(trial)
    early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks = [early_stopping])
    
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(optimal, n_trials=50)

print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2025-11-11 10:11:10,441] A new study created in memory with name: no-name-6b9e95c3-790f-48bc-97a2-503ff52d0a83
C:\Users\admin\AppData\Local\Temp\ipykernel_12352\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.6548 - loss: 0.2891
Epoch 2/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7273 - loss: 0.2559
Epoch 3/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7349 - loss: 0.2539
Epoch 4/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7387 - loss: 0.2527
Epoch 5/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7405 - loss: 0.2520
Epoch 6/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7422 - loss: 0.2514
Epoch 7/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7435 - loss: 0.2510
Epoch 8/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.7443 - loss: 0.2506
Epoch 9/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7453 - loss: 0.2504
Epoch 10/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7457 - loss: 0.2501
Epoch 11/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.7467 - loss: 0.2499
Epoch 12/22
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7472 - loss: 0.2498

[I 2025-11-11 10:14:07,626] Trial 0 finished with value: 0.7444090078813803 and parameters: {'epochs': 22, 'batch_size': 56, 'units_layer1': 7, 'units_layer2': 32, 'optimizer': 'adam', 'learning_rate': 0.0002454790046811478}. Best is trial 0 with value: 0.7444090078813803.


Epoch 1/32


C:\Users\admin\AppData\Local\Temp\ipykernel_12352\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


...

Epoch 1/34


C:\Users\admin\AppData\Local\Temp\ipykernel_12352\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


5592/5592 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.5487 - loss: 0.4690
Epoch 2/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.5486 - loss: 0.4012
Epoch 3/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - AUC: 0.5483 - loss: 0.3637
Epoch 4/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.5492 - loss: 0.3417
Epoch 5/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - AUC: 0.5508 - loss: 0.3281
Epoch 6/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - AUC: 0.5531 - loss: 0.3192
Epoch 7/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.5552 - loss: 0.3133
Epoch 8/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.5580 - loss: 0.3090
Epoch 9/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.5606 - loss: 0.3059
Epoch 10/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.5634 - loss: 0.3035
Epoch 11/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.5665 - loss: 0.3015
Epoch 12/34
5592/5592 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.5694 - loss: 0.2998
Epoch 13

[I 2025-11-11 10:54:11,195] Trial 9 finished with value: 0.6281408950350762 and parameters: {'epochs': 34, 'batch_size': 44, 'units_layer1': 28, 'units_layer2': 20, 'optimizer': 'sgd', 'learning_rate': 3.5972855659334694e-05}. Best is trial 0 with value: 0.7444090078813803.


Best trial: 0.7444090078813803
Best hyperparameters: {'epochs': 22, 'batch_size': 56, 'units_layer1': 7, 'units_layer2': 32, 'optimizer': 'adam', 'learning_rate': 0.0002454790046811478}


In [107]:
best_params = study.best_trial.params

best_params

{'epochs': 22,
 'batch_size': 56,
 'units_layer1': 7,
 'units_layer2': 32,
 'optimizer': 'adam',
 'learning_rate': 0.0002454790046811478}

In [109]:
# Train the final model with the best hyperparameters

best_model = Sequential()
best_model.add(Dense(units=best_params['units_layer1'], activation='relu'))
best_model.add(Dense(units=best_params['units_layer2'], activation='relu'))
best_model.add(Dense(1, activation='sigmoid'))

In [111]:
if best_params['optimizer'] == 'adam':
    best_optimizer = Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'sgd':
    best_optimizer = SGD(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'rmsprop':
    best_optimizer = RMSprop(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'adagrad':
    best_optimizer = Adagrad(learning_rate=best_params['learning_rate'])


In [113]:
best_model.compile(optimizer=best_optimizer, loss='binary_crossentropy', metrics=['AUC'])

In [115]:
def evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train, epochs=50, batch_size=best_params['batch_size'])
    
    '''Predictions and probabilities for the training set'''
    
    y_train_prob = model.predict(X_train)

    '''Predictions and probabilities for the test set'''
    
    y_test_prob = model.predict(X_test)

    '''Calculate metrics for the training set''' 
    
    roc_train_prob = roc_auc_score(y_train, y_train_prob)
    gini_train_prob = roc_train_prob * 2 - 1
    

    '''Calculate metrics for the test set'''
    
    roc_test_prob = roc_auc_score(y_test, y_test_prob)
    gini_test_prob = roc_test_prob * 2 - 1
    

    results = pd.DataFrame({
        'Dataset': ['Train', 'Test'],
        'Gini': [gini_train_prob * 100, gini_test_prob * 100],
    
    })

    return results

In [117]:
evaluate(best_model, X_train, y_train, X_test, y_test)

Epoch 1/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - AUC: 0.6536 - loss: 0.2940
Epoch 2/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - AUC: 0.7274 - loss: 0.2561
Epoch 3/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7350 - loss: 0.2535
Epoch 4/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - AUC: 0.7390 - loss: 0.2523
Epoch 5/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7414 - loss: 0.2515
Epoch 6/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7431 - loss: 0.2509
Epoch 7/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7445 - loss: 0.2506
Epoch 8/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7452 - loss: 0.2503
Epoch 9/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7463 - loss: 0.2500
Epoch 10/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7469 - loss: 0.2498
Epoch 11/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.7473 - loss: 0.2497
Epoch 12/23
4393/4393 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - AUC: 0.7477 - loss: 0.2495


,Dataset,Gini
0,Train,50.688438
1,Test,48.644477
